<a href="https://colab.research.google.com/github/harish-03-11/news-classification-spacy/blob/main/News_classification_Word_vectors_using_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy
import pandas as pd
import numpy as np

In [ ]:
df_true = pd.read_csv("/content/sample_data/True.csv")
df_fake = pd.read_csv("/content/sample_data/Fake.csv")

In [ ]:
df_true.shape


(21417, 4)

In [ ]:
df_fake.shape

(23481, 4)

In [ ]:
21417-2000

19417

In [ ]:
23481-2000

21481

In [ ]:
import random

num_rows_to_drop_true = 19417
random_indices_true = random.sample(range(len(df_true)), num_rows_to_drop_true)
df_true = df_true.drop(random_indices_true, axis=0)
df_true.shape

(2000, 4)

In [ ]:

num_rows_to_drop_fake = 21481
random_indices_fake = random.sample(range(len(df_fake)), num_rows_to_drop_fake)
df_fake = df_fake.drop(random_indices_fake, axis=0)
df_fake.shape

(2000, 4)

In [ ]:
df_true['label'] = 1

In [ ]:
df_fake['label'] = 0

In [ ]:
df_combined = pd.concat([df_true,df_fake],axis=0)
df_combined.shape

(4000, 5)

In [ ]:
df = df_combined.sample(frac=1)

In [ ]:
df.head(4)

,title,text,subject,date,label
12287,Undercover Israeli agents in action,"EL-BIREH, West Bank (Reuters) - Reuters photog...",worldnews,"December 14, 2017",1
18421,"Mattis plays down split between Trump, Tillers...",WASHINGTON (Reuters) - U.S. Defense Secretary ...,worldnews,"October 3, 2017",1
14621,Greeks in mourning and disbelief after flood t...,"MANDRA, Greece (Reuters) - Greeks voiced despa...",worldnews,"November 16, 2017",1
20472,KARMA? PODIUM COLLAPSES At Hillary Rally [Video],Hillary Clinton was all smiles at a rally in C...,left-news,"Jun 5, 2016",0


In [ ]:
df = df.drop('title',axis = 1)

In [ ]:
df.sample(3)

,text,subject,date,label
4899,WASHINGTON (Reuters) - U.S. Defense Secretary ...,politicsNews,"March 16, 2017",1
19461,VIENNA (Reuters) - A French businessman said h...,worldnews,"September 21, 2017",1
3131,WASHINGTON (Reuters) - The top Republican in t...,politicsNews,"June 19, 2017",1


In [ ]:
df = df.drop('subject',axis =1)
df.sample(3)

,text,date,label
8402,CHICAGO (Reuters) - Potential action this week...,"August 23, 2016",1
10745,The outspoken comedian posed for a photo shoot...,"May 30, 2017",0
9987,WASHINGTON (Reuters) - A U.S. Senate committee...,"April 11, 2016",1


In [ ]:
df = df.drop('date',axis =1)
df.head(4)

,text,label
12287,"EL-BIREH, West Bank (Reuters) - Reuters photog...",1
18421,WASHINGTON (Reuters) - U.S. Defense Secretary ...,1
14621,"MANDRA, Greece (Reuters) - Greeks voiced despa...",1
20472,Hillary Clinton was all smiles at a rally in C...,0


In [ ]:
!python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 842.1 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
df['vector'] = df['text'].apply(lambda x: nlp(x).vector)
df.sample(3)

,text,label,vector
22209,Andrew Korybko 21st Century Wire The global m...,0,"[-1.9458108, 1.056299, -1.8784553, 0.5085458, ..."
5083,Two women in New Jersey are suing the state fo...,0,"[-1.8273222, 1.6571219, -2.4409385, -0.1304925..."
16608,BAGHDAD (Reuters) - Iraqi pro-government param...,1,"[-2.3080378, 0.4071469, -1.976701, 1.3824251, ..."


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler

In [ ]:
X = df['vector']
y = df['label']

In [ ]:
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state=2022,stratify=y)

In [ ]:
X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [ ]:
scaler = MinMaxScaler()
scaler_Xtrain_embed = scaler.fit_transform(X_train_2d)
scaler_Xtest_embed = scaler.transform(X_test_2d)

In [ ]:
model = MultinomialNB()

In [ ]:
model.fit(scaler_Xtrain_embed,y_train)

MultinomialNB()

In [ ]:
y_pred = model.predict(scaler_Xtest_embed)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.82      0.90      0.86       365
           1       0.91      0.84      0.87       435

    accuracy                           0.87       800
   macro avg       0.87      0.87      0.87       800
weighted avg       0.87      0.87      0.87       800

